Cargar Data y primeros analisis

In [1]:
import bootcampviztools as bt
import hashlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import modulo_toolbox as mtb
import requests
import seaborn as sns
import time
import toolbox_ML as tb
import warnings

warnings.filterwarnings('ignore')

In [2]:
df_DC = pd.read_csv('./heroes_dc_extended')

In [3]:
df_DC.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       73 non-null     int64 
 1   nombre_completo  61 non-null     object
 2   nombre           73 non-null     object
 3   universo         73 non-null     object
 4   url              73 non-null     object
 5   Intelligence     73 non-null     int64 
 6   Strength         73 non-null     int64 
 7   Speed            73 non-null     int64 
 8   Durability       73 non-null     int64 
 9   Power            73 non-null     int64 
 10  Combat           73 non-null     int64 
 11  Tier             73 non-null     int64 
 12  Tier2            9 non-null      object
 13  heroe            73 non-null     object
dtypes: int64(8), object(6)
memory usage: 8.1+ KB


In [4]:
from itertools import combinations


# Crear combinaciones de pares de héroes
pairs = list(combinations(df_DC.index, 2))

# Crear un nuevo DataFrame para los pares de héroes y resultados de batallas
battle_data = []

for i, j in pairs:
    hero1 = df_DC.loc[i]
    hero2 = df_DC.loc[j]
       
   
    battle_data.append({
        'Heroe_1': hero1['nombre'],
        'Heroe_2': hero2['nombre'],
        
    })

In [5]:
df_batallas.head(50)

NameError: name 'df_batallas' is not defined

In [ ]:
df_batallas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3655 entries, 0 to 3654
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Heroe_1  3655 non-null   object
 1   Heroe_2  3655 non-null   object
dtypes: object(2)
memory usage: 57.2+ KB


In [ ]:
# Mostrar las primeras filas del dataframe para inspeccionar
df_DC.head(20)

,Unnamed: 0,nombre_completo,nombre,universo,url,Intelligence,Strength,Speed,Durability,Power,Combat,Tier,Tier2,heroe
0,0,Alfred Pennyworth,Alfred,DC Extended Universe,https://www.superherodb.com/alfred/10-14275/#shdb,65,1,1,5,1,20,0,NaN,Alfred
1,1,Amanda Blake Waller,Amanda Waller,DC Extended Universe,https://www.superherodb.com/amanda-waller/10-3...,65,1,1,5,10,30,0,NaN,Amanda Waller
2,2,NaN,Anthea,DC Extended Universe,https://www.superherodb.com/anthea/10-54768/#user,0,0,0,0,0,0,0,NaN,Anthea
3,3,Arthur Curry,Aquaman,DC Extended Universe,https://www.superherodb.com/aquaman/10-12533/,65,50,25,30,45,80,3,NaN,Aquaman
4,4,Ares,Ares,DC Extended Universe,https://www.superherodb.com/ares/10-12597/,90,100,50,50,70,100,4,4,Ares
5,5,Albert Rothstein,Atom Smasher,DC Extended Universe,https://www.superherodb.com/atom-smasher/10-27...,60,30,25,25,40,20,2,NaN,Atom Smasher
6,6,Bruce Wayne,Batman,DC Extended Universe,https://www.superherodb.com/batman/10-12530/,80,13,11,15,20,95,1,NaN,Batman
7,7,Teth-Adam,Black Adam,DC Extended Universe,https://www.superherodb.com/black-adam/10-21003/,65,100,75,45,75,75,3,3,Black Adam
8,8,Dinah Lance,Black Canary,DC Extended Universe,https://www.superherodb.com/black-canary/10-18...,55,1,6,5,35,40,1,NaN,Black Canary
9,9,David Kane,Black Manta,DC Extended Universe,https://www.superherodb.com/black-manta/10-15579/,70,25,25,20,40,60,2,NaN,Black Manta


Nuevo data con batallas

In [ ]:
from itertools import combinations

# Crear combinaciones de pares de héroes
pairs = list(combinations(df_DC.index, 2))

# Crear un nuevo DataFrame para los pares de héroes y resultados de batallas
battle_data = []

for i, j in pairs:
    hero1 = df_DC.loc[i]
    hero2 = df_DC.loc[j]
    fuerza1 = hero1['Fuerza_Total']
    fuerza2 = hero2['Fuerza_Total']
    
    # Determinar el resultado de la batalla
    if fuerza1 > fuerza2:
        resultado = 'Gana'
    elif fuerza1 < fuerza2:
        resultado = 'Pierde'
    else:
        resultado = 'Empata'
    
    battle_data.append({
        'Heroe_1': hero1['nombre'],
        'Fuerza_Heroe_1': fuerza1,
        'Heroe_2': hero2['nombre'],
        'Fuerza_Heroe_2': fuerza2,
        'Resultado': resultado
    })

In [ ]:
# Convertir la lista de batallas en un DataFrame
battle_df = pd.DataFrame(battle_data)

# Mostrar las primeras filas del DataFrame de batallas
battle_df.head(50)

,Heroe_1,Fuerza_Heroe_1,Heroe_2,Fuerza_Heroe_2,Resultado
0,Alfred,93,Amanda Waller,112,Pierde
1,Alfred,93,Anthea,0,Gana
2,Alfred,93,Aquaman,295,Pierde
3,Alfred,93,Ares,460,Pierde
4,Alfred,93,Atom Smasher,200,Pierde
5,Alfred,93,Batman,234,Pierde
6,Alfred,93,Black Adam,435,Pierde
7,Alfred,93,Black Canary,142,Pierde
8,Alfred,93,Black Manta,240,Pierde
9,Alfred,93,Black Mask,107,Pierde


Mapeo atributos heroes en dataframe batallas

In [ ]:
# Vamos a añadir las columnas de atributos para cada héroe
# hero_attributes = ['Intelligence', 'Strength', 'Speed', 'Durability', 'Power', 'Combat']

In [ ]:
# # Unir los atributos de Héroe_1
# for attr in hero_attributes:
#     battle_df[f'Heroe_1_{attr}'] = battle_df['Heroe_1'].map(df_DC.set_index('nombre')[attr])

# # Unir los atributos de Héroe_2
# for attr in hero_attributes:
#     battle_df[f'Heroe_2_{attr}'] = battle_df['Heroe_2'].map(df_DC.set_index('nombre')[attr])

In [ ]:
# # Convertir el resultado a valores numéricos
# result_mapping = {'Gana': 1, 'Pierde': 0, 'Empata': 2}
# battle_df['Resultado_Numerico'] = battle_df['Resultado'].map(result_mapping)

In [ ]:
battle_df.iloc[150:200]

,Heroe_1,Fuerza_Heroe_1,Heroe_2,Fuerza_Heroe_2,Resultado
150,Amanda Waller,112,Rick Flag,152,Pierde
151,Amanda Waller,112,Sabbac,410,Pierde
152,Amanda Waller,112,Savant,112,Empata
153,Amanda Waller,112,Shazam,355,Pierde
154,Amanda Waller,112,Slipknot,87,Gana
155,Amanda Waller,112,Sol Soria,107,Gana
156,Amanda Waller,112,Starro,275,Pierde
157,Amanda Waller,112,Steppenwolf,415,Pierde
158,Amanda Waller,112,Steve Trevor,117,Pierde
159,Amanda Waller,112,Supergirl,345,Pierde


In [ ]:
battle_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3655 entries, 0 to 3654
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Heroe_1         3655 non-null   object
 1   Fuerza_Heroe_1  3655 non-null   int64 
 2   Heroe_2         3655 non-null   object
 3   Fuerza_Heroe_2  3655 non-null   int64 
 4   Resultado       3655 non-null   object
dtypes: int64(2), object(3)
memory usage: 142.9+ KB


In [ ]:
# battle_df.to_csv('batallas_Dc')